# Spotify Roblox API Project

In [ ]:
import configparser
import spotipy
from utils.auth_init import auth_init
import utils.spotify_functions as spotify_functions
import utils.roblox_functions as roblox_functions
import utils


In [2]:
# Check key folder and if does not exist create empty templates for keys
auth_init()

auth/spotify_keys.ini already exists.
auth/roblox_keys.ini already exists.


## Code to Create Spotify Playlist Based on User

In [ ]:
import robloxpy
import requests
import webbrowser
import spotipy
import spotipy.util as util
import json
import base64
from roblox import Client
import roblox.thumbnails as thumbnail
import asyncio
from utils.roblox_functions import *
import utils.spotify_functions as spotify_functions
import spotipy
import pandas as pd 
from spotipy.oauth2 import SpotifyClientCredentials


In [ ]:
# Get Spotify Authentication
keys_path = "auth/spotify_keys.ini"
spotify_auth = spotify_functions.SpotifyAuth(keys_path)

# Authenticate Spotify and get credentials manager as sp
client_credentials_manager = SpotifyClientCredentials(spotify_auth.client_id, spotify_auth.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Spotify scopes (INCLUDE ugc-image-upload)
scope = "user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public playlist-modify-private ugc-image-upload"
token = util.prompt_for_user_token(spotify_auth.username, scope, client_id=spotify_auth.client_id, client_secret=spotify_auth.client_secret, redirect_uri=spotify_auth.redirectURI)

sp = spotipy.Spotify(auth=token)

In [ ]:
# Read Roblox Cookie to get specific user
RobloxAuth = RobloxAuthConfig('auth/roblox_keys.ini')
cookie = RobloxAuth.cookie

# Setting Roblox Cookie to Client
client = Client(cookie)

async def main():
    user = await client.get_authenticated_user()
    # Test to see if you are in
    # *** ID CAN BE PRINTED: it is found in Roblox profile URL***
    print("ID:", user.id)
await main()

ID: 7520736841


In [ ]:
# Input desired Roblox Username
desired_name = "WHATEVER NAME YOU WANT"
outputted_names = []

async def main():
    # Search the username and X results
    users = client.user_search(desired_name, max_items =10)
    async for user in users:
        # Add to list
        outputted_names.append(user.name)
        outputted_names.append(user.id)
        
await main()

formatted_names = []
for i in range(0, len(outputted_names), 2):
    # Format list to dictonary to access Name & ID
    formatted_names.append({'name': outputted_names[i], 'id': outputted_names[i + 1]})

formatted_names

[{'name': 'Roblox', 'id': 1},
 {'name': 'roblox_pro12163', 'id': 7418004099},
 {'name': 'RobloxRealAccount991', 'id': 7540027439},
 {'name': 'lukepukenow', 'id': 5033340271},
 {'name': '2019Squish', 'id': 4370141275},
 {'name': '19ewhospayzzz86', 'id': 7520073920},
 {'name': 'Zeph123911', 'id': 7526353872},
 {'name': 'roblox_556744', 'id': 3175414490},
 {'name': 'RobloxRobloxYT_1', 'id': 7148111849},
 {'name': 'Roblox0826135', 'id': 7531479514}]

In [ ]:
# Pick a user: this could be random but in this case its top result
desired_user = formatted_names[0]
user = await client.get_user(desired_user["id"])
# Get the avatar thumbnaill found on their profile
user_thumbnails = await client.thumbnails.get_user_avatar_thumbnails(
    users=[user],
    type=thumbnail.AvatarThumbnailType.full_body,
    size=(420, 420)
)

# Print the URL of user image
if len(user_thumbnails) > 0:
    user_thumbnail = user_thumbnails[0]
    user_image = user_thumbnail.image_url
    print('Url of user image:')
    print(user_image)

https://tr.rbxcdn.com/30DAY-Avatar-310966282D3529E36976BF6B07B1DC90-Png/420/420/Avatar/Png/noFilter


In [ ]:
# Search Spotify based on the Robloxians Username
track_results = sp.search(q=desired_user["name"], type="track", limit = 10)
# print(track_results)

In [ ]:
# function to clean dataframe in spotify_functions.py - takes in track results, clean into a nice dataframe
clean_spotify_df = spotify_functions.spotify_tracks_to_df(track_results)
clean_spotify_df

,song,artists
0,Don't Call Me a Noob,Mr Miln
1,Skibidi Toilet,Lil Big Stack
2,Slaying in Roblox,Loginhdi
3,The Roblox Song,Lankybox
4,Roblox NEW DEATH SOUND Song,Misutra
5,Robloxian Struggles (Playing Without Robux),Mr Miln
6,Relaxed Scene,James Clarke
7,Friday Theme,Uglyburger0
8,Monday Theme,Uglyburger0
9,Adopt Me! Night Theme,Club Unicorn


## Code to Parse Roblox Lua

In [ ]:
import glob

# get all lua paths
directory = 'lua_scripts/*'

lua_file_paths = glob.glob(directory)
lua_file_paths


['lua_scripts\\display_script.lua',
 'lua_scripts\\parameter_classes.lua',
 'lua_scripts\\ui_script.lua']

In [ ]:
import re
# can move this to script file but leaving it hear for you to see regex

def sub_song_artist_lua(clean_spotify_df, lua_param_file_path):
    """Takes in spotify clean dataframe and lua parameter file,
    return substituted code using regex """

    # Read the Lua script from the file
    file_path = lua_param_file_path
    with open(file_path, 'r') as file:
        lua_script = file.read()

    text = lua_script

    # Regex pattern to match all song entries - artists = and song =
    pattern = r'{artist = "([^"]+)", song = "([^"]+)"}'

    # Find all matches
    matches = list(re.finditer(pattern, text))

    # Substitute each match one by one
    for i,match in enumerate(matches):
        artist, song = match.groups()
        print(f"Matched artist: {artist}, song: {song}")
        
        # Modify text: replace current match with a custom substitution
        text = re.sub(re.escape(match.group(0)), f'{{artist = "{clean_spotify_df['artists'][i]}", song = "{clean_spotify_df['song'][i]}"}}', text, count=1)

    print("\nModified Text:\n")
    print(text)

    return text

def read_lua(file_path):
    # Read the Lua script from the file
    with open(file_path, 'r') as file:
        lua_script = file.read()

    text = lua_script

    return text





Matched artist: Artist 1, song: Song 1
Matched artist: Artist 2, song: Song 2
Matched artist: Artist 3, song: Song 3
Matched artist: Artist 4, song: Song 4
Matched artist: Artist 5, song: Song 5
Matched artist: Artist 6, song: Song 6
Matched artist: Artist 7, song: Song 7
Matched artist: Artist 8, song: Song 8
Matched artist: Artist 9, song: Song 9
Matched artist: Artist 10, song: Song 10

Modified Text:

-- ParameterStart
local data = {
	{artist = "Mr Miln", song = "Don't Call Me a Noob"},
	{artist = "Lil Big Stack", song = "Skibidi Toilet"},
	{artist = "Loginhdi", song = "Slaying in Roblox"},
	{artist = "Lankybox", song = "The Roblox Song"},
	{artist = "Misutra", song = "Roblox NEW DEATH SOUND Song"},
	{artist = "Mr Miln", song = "Robloxian Struggles (Playing Without Robux)"},
	{artist = "James Clarke", song = "Relaxed Scene"},
	{artist = "Uglyburger0", song = "Friday Theme"},
	{artist = "Uglyburger0", song = "Monday Theme"},
	{artist = "Club Unicorn", song = "Adopt Me! Night Theme"}

In [ ]:
# call function to substitute parameters in lua template with new spotify song and artist names
sub_params_lua_str = sub_song_artist_lua(clean_spotify_df,'lua_scripts\\parameter_classes.lua')
display_script_lua = read_lua('lua_scripts\\display_script.lua')
ui_script_lua = read_lua('lua_scripts\\ui_script.lua')

# Add to other scripts to make complete script, \n required to keep lua formatting
complete_display_script_lua = sub_params_lua_str+"\n"+display_script_lua
complete_ui_script_lua = sub_params_lua_str+"\n"+ui_script_lua
